In [41]:
# pip install lyricsgenius

In [2]:
from lyricsgenius import Genius

genius = Genius(access_token='LTWI3Y78oDOqchjeB0VjDD7pZWGXqrle45YPv8QIcyzQtMWPMg6RHqbXfWleen6D',
                timeout=100)
genius.remove_section_headers = True

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from tqdm.notebook import tqdm

In [4]:
def get_top_100(soup):
    songs = [x.text.strip() for x in soup.select('li .c-title')]
    artists = [x.text.strip() for x in soup.select('li .c-title ~ .c-label')]
    return {rank:{'song':song, 'artist':artist} for rank, (song, artist) in enumerate(zip(songs,artists), start=1)}

In [5]:
# proxies = {'http': 'http://206.189.157.23'}

# years = range(2006,2022)
# song_data = {}
# for year in years:
#     url = f'https://www.billboard.com/charts/year-end/{year}/hot-100-songs/'
#     soup = BeautifulSoup(requests.get(url, proxies=proxies).content)
#     song_data[year] = get_top_100(soup)

In [7]:
# df_all_years = pd.DataFrame()
# for year in years:
#     df_year = pd.DataFrame(song_data[year]).T.reset_index().rename(columns={'index':'rank'})
#     df_year['year'] = year
#     df_all_years = pd.concat([df_all_years, df_year])
# df_all_years

In [9]:
# df_all_years.to_pickle("all_songs.pkl")

In [40]:
df_all_years = pd.read_pickle('all_songs.pkl')
df_all_years

,rank,song,artist,year
0,1,Bad Day,Daniel Powter,2006
1,2,Temperature,Sean Paul,2006
2,3,Promiscuous,Nelly Furtado Featuring Timbaland,2006
3,4,You're Beautiful,James Blunt,2006
4,5,Hips Don't Lie,Shakira Featuring Wyclef Jean,2006
...,...,...,...,...
95,96,Things A Man Oughta Know,Lainey Wilson,2021
96,97,Throat Baby (Go Baby),BRS Kash,2021
97,98,Tombstone,Rod Wave,2021
98,99,Drinkin' Beer. Talkin' God. Amen.,Chase Rice Featuring Florida Georgia Line,2021


In [11]:
import re

def preprocess_lyrics(lyrics):
    
    # Convert to lowercase
    lyrics = lyrics.lower()
    
    # Removes titles
    lyrics = lyrics.split('yrics\n', 1)[-1]
    
    # Removes 'Embed from the end'
    lyrics = lyrics.rsplit('embed', 1)[0]
    
    # Removes digits before embed
    lyrics = lyrics.rstrip('1234567890')
    
    # Removes 'you might also like'
    lyrics = lyrics.rsplit('you might also like', 1)[0]
    
    # Replace dash with space
    lyrics = (re.sub("[-]", " ", lyrics))
    
    # Remove Non-letters
    lyrics = (re.sub("[^\sA-Za-z]", "", lyrics))
    
    # Standardize whitespace
    lyrics = (re.sub("[\s]", " ", lyrics))
    
    return lyrics

In [12]:
def preprocess_query(query):
    
    # Convert to lowercase
    query = query.lower()
    
    # Remove at end
    query = query.rsplit('featuring', 1)[0]
    query = query.rsplit(' x', 1)[0]
    query = query.rsplit(' /', 1)[0]
    
    # Replace
    query = (re.sub("the black", "black", query))
    query = (re.sub("f\*\*k", "fuck", query))
    query = (re.sub("ni\*\*a", "nigga", query))
    query = (re.sub("b\*\*\*\*", "bitch", query))
    query = (re.sub("part 2", "pt 2", query))
    query = (re.sub("p\*\$\$y", "pussy", query))
    query = (re.sub("ily surf", "ily (i love you baby) surf", query))
    query = (re.sub("\(spider-man: into the spider-verse\)", "", query))
    query = (re.sub("\(fifty shades darker\)", "", query))
    query = (re.sub("it ain't me", "it aint me", query))
    query = (re.sub("tik tok ke\$ha", "tik tok kesha", query))
    query = (re.sub("jay-z, rihanna", "jayz", query))
    
    return query

In [13]:
def get_lyrics(query):
    song_id = genius.search_songs(f"{query}",
                                  per_page=1,
                                  page=1)['hits'][0]['result']['id']
    print(f'Got lyrics from query: {query}')
    return genius.lyrics(song_id)

In [14]:
df_test = df_all_years.copy()
df_test['query'] = (df_test['song'] + ' ' + df_test['artist'].apply(lambda x: " ".join(x.split()[:2])))
df_test['query'] = df_test['query'].apply(preprocess_query)

In [16]:
# df_2006 = df_test[df_test['year']==2006].copy()
# df_2007 = df_test[df_test['year']==2007].copy()
# df_2008 = df_test[df_test['year']==2008].copy()
# df_2009 = df_test[df_test['year']==2009].copy()
# df_2010 = df_test[df_test['year']==2010].copy()
# df_2011 = df_test[df_test['year']==2011].copy()
# df_2012 = df_test[df_test['year']==2012].copy()
# df_2013 = df_test[df_test['year']==2013].copy()
# df_2014 = df_test[df_test['year']==2014].copy()
# df_2015 = df_test[df_test['year']==2015].copy()
# df_2015 = df_test[df_test['year']==2015].copy()
# df_2016 = df_test[df_test['year']==2016].copy()
# df_2017 = df_test[df_test['year']==2017].copy()
# df_2018 = df_test[df_test['year']==2018].copy()
# df_2019 = df_test[df_test['year']==2019].copy()
# df_2020 = df_test[df_test['year']==2020].copy()
# df_2021 = df_test[df_test['year']==2021].copy()

# df_2006

In [17]:
df_2006 = pd.read_pickle('songs_2006.pkl')
df_2007 = pd.read_pickle('songs_2007.pkl')
df_2008 = pd.read_pickle('songs_2008.pkl')
df_2009 = pd.read_pickle('songs_2009.pkl')
df_2010 = pd.read_pickle('songs_2010.pkl')
df_2011 = pd.read_pickle('songs_2011.pkl')
df_2012 = pd.read_pickle('songs_2012.pkl')
df_2013 = pd.read_pickle('songs_2013.pkl')
df_2014 = pd.read_pickle('songs_2014.pkl')
df_2015 = pd.read_pickle('songs_2015.pkl')
df_2015 = pd.read_pickle('songs_2015.pkl')
df_2016 = pd.read_pickle('songs_2016.pkl')
df_2017 = pd.read_pickle('songs_2017.pkl')
df_2018 = pd.read_pickle('songs_2018.pkl')
df_2019 = pd.read_pickle('songs_2019.pkl')
df_2020 = pd.read_pickle('songs_2020.pkl')
df_2021 = pd.read_pickle('songs_2021.pkl')

In [58]:
# df_2006['lyrics'] = df_2006['query'].apply(get_lyrics)
# df_2006.to_pickle('songs_2006.pkl')

# df_2007['lyrics'] = df_2007['query'].apply(get_lyrics)
# df_2007.to_pickle('songs_2007.pkl')

# df_2008['lyrics'] = df_2008['query'].apply(get_lyrics)
# df_2008.to_pickle('songs_2008.pkl')

# df_2009['lyrics'] = df_2009['query'].apply(get_lyrics)
# df_2009.to_pickle('songs_2009.pkl')

# df_2010['lyrics'] = df_2010['query'].apply(get_lyrics)
# df_2010.to_pickle('songs_2010.pkl')

# df_2011['lyrics'] = df_2011['query'].apply(get_lyrics)
# df_2011.to_pickle('songs_2011.pkl')

# df_2012['lyrics'] = df_2012['query'].apply(get_lyrics)
# df_2012.to_pickle('songs_2012.pkl')

# df_2013['lyrics'] = df_2013['query'].apply(get_lyrics)
# df_2013.to_pickle('songs_2013.pkl')

# df_2014['lyrics'] = df_2014['query'].apply(get_lyrics)
# df_2014.to_pickle('songs_2014.pkl')

# df_2015['lyrics'] = df_2015['query'].apply(get_lyrics)
# df_2015.to_pickle('songs_2015.pkl')

# df_2016['lyrics'] = df_2016['query'].apply(get_lyrics)
# df_2016.to_pickle('songs_2016.pkl')

# df_2017['lyrics'] = df_2017['query'].apply(get_lyrics)
# df_2017.to_pickle('songs_2017.pkl')

# df_2018['lyrics'] = df_2018['query'].apply(get_lyrics)
# df_2018.to_pickle('songs_2018.pkl')

# df_2019['lyrics'] = df_2019['query'].apply(get_lyrics)
# df_2019.to_pickle('songs_2019.pkl')

# df_2020['lyrics'] = df_2020['query'].apply(get_lyrics)
# df_2020.to_pickle('songs_2020.pkl')

# df_2021['lyrics'] = df_2021['query'].apply(get_lyrics)
# df_2021.to_pickle('songs_2021.pkl')

In [57]:
df_2010.loc[5]

rank                                                      6
song                                              Airplanes
artist                      B.o.B Featuring Hayley Williams
year                                                   2010
query                             Airplanes B.o.B Featuring
lyrics    TranslationsFrançaisAirplanes Lyrics\nCan we p...
Name: 5, dtype: object

In [59]:
df_2010

,rank,song,artist,year,query,lyrics
0,1,TiK ToK,Ke$ha,2010,TiK ToK Ke$ha,TranslationsPortuguêsTiK ToK Lyrics\nWake up i...
1,2,Need You Now,Lady Antebellum,2010,Need You Now Lady Antebellum,"Need You Now Lyrics\n""Hey, sorry I missed your..."
2,3,"Hey, Soul Sister",Train,2010,"Hey, Soul Sister Train","Hey, Soul Sister Lyrics\nHey\nHe-e-e-e-ey\nHe-..."
3,4,California Gurls,Katy Perry Featuring Snoop Dogg,2010,California Gurls Katy Perry,TranslationsEspañolPortuguêsCalifornia Gurls L...
4,5,OMG,Usher Featuring will.i.am,2010,OMG Usher Featuring,OMG Lyrics\nOh my gosh\nBaby let me\nI did it ...
...,...,...,...,...,...,...
95,96,Life After You,Daughtry,2010,Life After You Daughtry,Life After You Lyrics\nTen miles from town and...
96,97,Smile,Uncle Kracker,2010,Smile Uncle Kracker,Smile Lyrics\nYou're better than the best\nI'm...
97,98,Teach Me How To Dougie,Cali Swag District,2010,Teach Me How To Dougie Cali Swag,"Teach Me How to Dougie Lyrics\nAy, ay\nTeach m..."
98,99,Try Sleeping With A Broken Heart,Alicia Keys,2010,Try Sleeping With A Broken Heart Alicia Keys,Try Sleeping with a Broken Heart Lyrics\nLet's...


In [ ]:
# Fixing invalid queries

# df_2007.loc[1, 'lyrics'] = get_lyrics('Umbrella Rihanna')
# df_2008.loc[37,'query'] = 'Bust It Baby Pt 2 Plies Featuring'
# df_2009.loc[30,'query'] = 'Run This Town JayZ'
# df_2009.loc[0, 'lyrics'] = get_lyrics('Boom Boom Pow Black')
# df_2009.loc[3, 'lyrics'] = get_lyrics('I Gotta Feeling Black')
# df_2010.loc[74,'query'] = 'Rock That Body Black Eyed Peas'
# df_2010.loc[0, 'lyrics'] = get_lyrics('TiK ToK Kesha')
# df_2010.loc[5, 'lyrics'] = get_lyrics('Airplanes B.o.B')
# df_2010.loc[10, 'lyrics'] = get_lyrics('Nothin On You B.o.B')
# df_2010.loc[62, 'lyrics'] = get_lyrics('Magic B.o.B')
# df_2010.loc[6, 'lyrics'] = get_lyrics('Love The Way You Lie Eminem')
# df_2010.loc[19, 'lyrics'] = get_lyrics('Imma be')
# df_2010.loc[28, 'lyrics'] = get_lyrics('I Gotta Feeling')
# df_2010.loc[57, 'lyrics'] = get_lyrics('Meet Me Halfway')
# df_2011.loc[17, 'query'] = 'Fuckin Perfect P!nk'
# df_2011.loc[1, 'lyrics'] = get_lyrics('Party Rock Anthem LMFAO')
# df_2012.loc[28, 'lyrics'] = get_lyrics('Party Rock Anthem LMFAO')
# df_2012.loc[39, 'lyrics'] = get_lyrics('Niggas in Paris JayZ')
# df_2013.loc[40, 'query'] = 'Fuckin Problems A$AP Rocky'
# df_2014.loc[34, 'query'] = 'Drunk In Love Beyonce'
# df_2014.loc[15, 'lyrics'] = get_lyrics('The Monster Eminem')
# df_2015.loc[47, 'lyrics'] = get_lyrics('Bitch Better Have My Money Rihanna')
# df_2016.loc[18, 'query'] = 'Me, Myself & I G-Eazy'
# df_2016.loc[3, 'lyrics'] = get_lyrics('Work Rihanna')
# df_2016.loc[10, 'lyrics'] = get_lyrics('Cheap Thrills Sia')
# df_2016.loc[33, 'lyrics'] = get_lyrics('Broccoli D.R.A.M.')
# df_2016.loc[59, 'lyrics'] = get_lyrics('Never Be Like You Flume')
# df_2016.loc[81, 'lyrics'] = get_lyrics('Pop Style Drake')
# df_2017.loc[5, 'lyrics'] = get_lyrics('Bad And Boujee')
# df_2017.loc[25, 'lyrics'] = get_lyrics('I Dont Wanna Live Forever')
# df_2017.loc[26, 'lyrics'] = get_lyrics('It Aint Me Kygo')
# df_2017.loc[69, 'lyrics'] = get_lyrics('Broccoli D.R.A.M.')
# df_2019.loc[1, 'lyrics'] = get_lyrics('Sunflower')
# df_2020.loc[4, 'lyrics'] = get_lyrics('Rockstar DaBaby')
# df_2020.loc[40, 'lyrics'] = get_lyrics('Laugh Now Cry Later Drake')
# df_2020.loc[93, 'lyrics'] = get_lyrics('Pussy Fairy (OTW) Jhene Aiko')
# df_2020.loc[94, 'lyrics'] = get_lyrics('ily (i love you baby) Surf Mesa')
# df_2021.loc[44, 'lyrics'] = get_lyrics('Laugh Now Cry Later Drake')
# df_2021.loc[47, 'lyrics'] = get_lyrics('Way 2 Sexy Drake')
# df_2021.loc[59, 'lyrics'] = get_lyrics('Essence Wizkid')
# df_2021.loc[62, 'lyrics'] = get_lyrics('Leave Before You Love Me Marshmello')
# df_2021.loc[87, 'lyrics'] = get_lyrics('just the way parmalee')